In [5]:
% load_ext autoreload
% autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Customer Segmentation of "Rate Beer" using Knowledge Graph Embeddings
This notebook uses Pytorch to create a Knowledge Graph Embedding of the customers purchases, usually data scientists try to segment customers in sequences like this using LSTM neural networks.

In [6]:
from clustering.rate_beer_loader import RateBeerLoaderPykeen

In the `RateBeerLoaderPykeen` object the `ratebeer.txt` file is read and creates a triples

In [7]:
pykeen_rate_beer = RateBeerLoaderPykeen("ratebeer.txt", checkpoint_name="torch_pipeline_checkpoints.pt")

Found a previous save
Loading Triples from the path downloaded.


In [8]:
training_input = pykeen_rate_beer.get_rate_beer()

Now all the training input data is within `training_input`.
We want a trained network embedding but we also want to be able to have scores as to the value of the graph.
The paper uses a list of

In [9]:
from pykeen.metrics.ranking import InverseHarmonicMeanRank
from pykeen.pipeline import pipeline
from pykeen.models import TransE
from pykeen.evaluation.rank_based_evaluator import RankBasedEvaluator

train_transe, test_transe = training_input.split(0.95)

transe_model_pipeline_results = pipeline(training=train_transe, testing=test_transe,
                                         training_loop='sLCWA', model=TransE,
                                         model_kwargs={"embedding_dim": 50},
                                         training_kwargs=dict(checkpoint_name="torch_pipeline_checkpoints.pt",
                                                              checkpoint_frequency=1, num_epochs=8),
                                         evaluator=RankBasedEvaluator,
                                         evaluator_kwargs=dict(metrics=[InverseHarmonicMeanRank], add_defaults=False))


using automatically assigned random_state=1693226516
INFO:pykeen.training.training_loop:=> loading checkpoint '/home/sora4222/.data/pykeen/checkpoints/torch_pipeline_checkpoints.pt'
INFO:pykeen.training.training_loop:=> loaded checkpoint '/home/sora4222/.data/pykeen/checkpoints/torch_pipeline_checkpoints.pt' stopped after having finished epoch 8
INFO:pykeen.stoppers.stopper:=> loading stopper summary dict from training loop checkpoint in '/home/sora4222/.data/pykeen/checkpoints/torch_pipeline_checkpoints.pt'
INFO:pykeen.stoppers.stopper:=> loaded stopper summary dictionary from checkpoint in '/home/sora4222/.data/pykeen/checkpoints/torch_pipeline_checkpoints.pt'


Training epochs on cuda:0: 100%|##########| 8/8 [00:00<?, ?epoch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=4.


Evaluating on cuda:0:   0%|          | 0.00/1.76M [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 22559.72s seconds


In [10]:
transe_model_pipeline_results.save_to_directory("transe_trained")

INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=3063968, num_relations=13, create_inverse_triples=False, num_triples=33384705, path="/mnt/a/PycharmProjects/UnsupervisedCustomerClustering/training_file.tsv") to file:///mnt/a/PycharmProjects/UnsupervisedCustomerClustering/transe_trained/training_triples
INFO:pykeen.pipeline.api:Saved to directory: file:///mnt/a/PycharmProjects/UnsupervisedCustomerClustering/transe_trained
